In [1]:
intention = '''Create an easy/safe API to parse/hold/yield constructions of pydantic docx class objects.
In theory this can be generalized and abstracted since the classes are largely absracted. 
However since this is API based, I will just focus on solutions for THIS dictionary task. These can be expanded later, or just have other APIs added
Final objective is have a tool to pass complex commands/instructions, and have it yield a pandas dataframe of the results.
Intermediate steps may be yield a list of lists
I am not yet sure if there is benefit trying to have the whole dictionary in one notional object. 
I don't think so, since each operation is psuedo atomic, and any aggregate actions are intended to be done in something like pandas.
Eventually I would want a loop for pandas operations to easily feed updates to the data. 
'''
conclusion = '''create a pular dictionary specifc datastructure to handle the aggregate paragraphs. 
This will only be per lemma, 
as any aggregations from roots and relations of that sort can be managed in pandas and created by simple inheretance/reference
'''

In [12]:
from typing import Optional, Dict, List, Any, Union, Tuple
import pydantic
from pydantic import ValidationError, validator, root_validator, Field, constr
from pydantic_docx import Docx_Paragraph_and_Runs, read_docx #type:ignore
from pydantic_docx_processor import create_sized_dataframe, expand_dataframe #type:ignore
import re
import json
from itertools import compress, chain
from datetime import datetime
import pandas as pd
import numpy as np
from functools import partial

c:\Users
c:\


In [3]:
import pickle
with open('pickled_results/feature_Frames_and_Indexes.pkl', 'rb') as file:
    # Call load method to deserialze
    output = pickle.load(file, encoding='utf-8')
(
parsed_object_list, #:List[Tuple[int,Docx_Paragraph_and_Runs]]
parsed_object_lookup, #:Dict[int,Docx_Paragraph_and_Runs] = dict(parsed_object_list)
doc_para_count, #: int = int(parsed_to_dict['total_encountered_paragraphs']) #type: ignore
char_counts, #: Counter = parsed_to_dict['char_counts'] #type: ignore
rootFrame,#:pd.DataFrame
rootsubpieceFrame, #:pd.DataFrame
lemmaFrame, #:pd.DataFrame
nonentityParaFrame, #pd.DataFrame
cleanerOutcomesDf #pd.DataFrame
    ) = output
paratext_lookup = {k:v.interogate__para_text() for k,v in parsed_object_lookup.items()}

## Handling cleaner fails and Performing Further Cleaning

In [4]:
#first check the frame containing the success/failure boolean for the cleaner operation that was performed on the data in pydantic_docx ingestion
# print(cleanerOutcomesDf['cleaner_success_outcomes'].unique())
# print(cleanerOutcomesDf.groupby('cleaner_success_outcomes').count())
cleanerOutcomesDf.dropna(subset=['paragraphIndex'],inplace=True)
# print(cleanerOutcomesDf['cleaner_success_outcomes'].unique())
# print(cleanerOutcomesDf.groupby('cleaner_success_outcomes').count())
# print(cleanerOutcomesDf[cleanerOutcomesDf['cleaner_success_outcomes']==False])
cleanerfailedInds = list(cleanerOutcomesDf[cleanerOutcomesDf['cleaner_success_outcomes']==False].index)
print(cleanerfailedInds)


all_para_text_frame = pd.Series(paratext_lookup).to_frame()
all_para_text_frame = expand_dataframe(all_para_text_frame,doc_para_count)
spread = 10
window_of_inds = [list(range(i-spread,i+spread)) for i in cleanerfailedInds]

# print(len(nonentityParaFrame.index.intersection(cleanerfailedInds))) #>>> 7, so all were captured in the nonentityParaFrame
print(nonentityParaFrame.shape)
nonentityParaFrame.drop(cleanerfailedInds, axis = 'index', inplace=True)
print(nonentityParaFrame.shape)
# all_para_text_frame.loc[window_of_inds[6]]

#results: 

# [1982, 1983] #>

   # all_para_text_frame.loc[window_of_inds[0]]
      # 1981	NaN	
      #># 1982	\t\t\t\t\t\t	
      #># 1983	\t\t\t\t\t\t\t\t	
      # 1984	NaN	
      # 1985	NaN	
      # 1986	Ɓ	
# 5971 #>
   # all_para_text_frame.loc[window_of_inds[2]]
      #># 5971	\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t
      # 5972	FAKO Sonr	
      # 5973	fako ? n. o	 
# 9318 #>
   # all_para_text_frame.loc[window_of_inds[3]]
      # 9316	GUYAAJI guyaaji, guyaale, guyaali pl.- nguyk...	
      # 9317	NaN	
      #># 9318	\t	
      # 9319	NaN	
      # 9320	NaN	
      # 9321	H	
      # 9322	NaN	
# [9786, 9787] #>
   # all_para_text_frame.loc[window_of_inds[4]]
      # 9784	non-castrated male, stallion, bull	
      # 9785	mâle non castré(D), étalon(Z), taureau(Z)	
      #># 9786	\t\t	
      #># 9787	\t\t\t	
      # 9788	HALƁ-	
      # 9789	halɓude v.av DCZ CZ<FJ,M,NO> [haacude] cf...	
# 20565 #>
   # all_para_text_frame.loc[window_of_inds[6]]
      # 20564	NaN	20564
      #># 20565	\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t	20565
      # 20566	Ŋ	20566
      # 20567	NaN	20567	

# all values with a valid paragraph will have a value in 'paragraphIndex'. ie paragraphs that did not have a suppressed error during class assignment. By default these are lines with only '\n'.
#when parsing, the default was used, so any all whitespace paragraph will be nan in paragraphIndex.
#when these values are dropped, the boolean column (indicating the successful post-processing of already created pydantic_docx class objects) yields True/False values.
# 7 values were false, as shown above. Each was a line of only '\t'. Lines with only '\n' are not even read into the model.
# adding control for this into the class object validation requirements may be a good idea. But better yet would be to pass a config with regex or simple roots to discard lines of a certain type. I'll add that to #TODO
# we can safely drop these records.
# ...and therefor, all are "good" for the next steps

[1982, 1983, 5971, 9318, 9786, 9787, 20565]
(17241, 1)
(17234, 1)


The following merge routine merges together the parents of one level of the heirarchy with their children. 
The parents then have their values forward-filled / duplicated to account for their headship over the child entities
Each frame has an identity column with a boolean value indicating that index is the location of that entity.
These are not needed for propogating headship and or child relationships, and these will not be forward-filled.
As such, these will be removed from the entity frames before they are added to the entity heirarchy frame

Precedence is as follows:
root > subroot > lemmas > paragraphs

However, the child relationship is non-exclusive to direct 'parents', such that a parent 'entity' may have a dhild from any of the levels below them.
That said, the dictionary description and general structure would indicate paragraphs should only be direct children of Lemmas
>paragraphs being defined as any line without an 'entity' such as root, subroot, or lemma.
>any that are not would be a deviation. 
Therefor, any such instance needs to be captured and assessed, and should not be coerced into some other neighbors "family" by default, if at all.

As a result, our routine for creating hierarchys must allow for any possible/permitted parent be available to match any children.

So...

Subroots:      possible parents are:      Roots
Lemmas:        possible parents are:      Roots, Subroots
Paragraphs:    possible parents are:      Roots, Subroots, Lemmas

*Note: the term 'subroot' is sometimes described as 'root-subpiece', and these names may be understood interchangibly. The reason for the two names stems from two competing 'theories' or impressions of the nature of these entities. Initially I used the term subroot as a description of their subordinate position. The term subpiece has begun entering my vocabulary after discovering that many (if not all) of these are 'infixes', a core component of the Pular language. As a result any of these that are infixes are more than just a manual 'byte pair encoding' or common break point, but have specific meaning of their own.*

In [5]:
all_entity_identity_frame = rootFrame[['is']].merge(rootsubpieceFrame[['is']],how='outer', on='index').merge(lemmaFrame[['is']],how='outer',on='index')
all_entity_identity_frame.columns = ['is_root', 'is_root_subpiece','is_lemma']

# note, the combine first method will remove any lemmas that are declared on the same line as the root
# rootFrame contains the permitted parents for subroots.
# print(rootFrame)
rootFrame['is']='root'
root_parent = rootFrame.copy()
#next the frame of permitted parents for lemmas is created by combining/overlaying roots and subroots
rootsubpieceFrame['is'] = 'root_subpiece'
any_roots = root_parent.combine_first(rootsubpieceFrame)

#finally create the permitted parents of paragraphs (though we will want to examine any paragraphs subordinate directly to a root. These may indicate some datacleaning is needed, or may need to be removed entirely)
lemmaFrame['is'] = 'lemma'
anyEntityFrame = any_roots.combine_first(lemmaFrame)

anyEntityFrame.sample(10)

,paragraphIndex,is,text,mask,run_text_list
index,,,,,
27840,27840.0,root_subpiece,-OR-,[True],[-OR-]
13901,13901.0,lemma,kabaaru,"[True, False, False]","[kabaaru , (ndu,o) / kabaaruuji (ɗi) , DFcZ ..."
12643,12643.0,lemma,"jaŋngo, janngo","[True, False, False]","[jaŋngo, janngo , n. , DFCZH CZ<FJ,FT,M,G,B..."
14070,14070.0,lemma,kankasi,"[True, False, False]","[kankasi , n. , T [mbudu, jaka]]"
22220,22220.0,lemma,peɗal,"[True, False, False]","[peɗal , (ngal) , D]"
21739,21739.0,lemma,"ñorngo, ñoro","[True, False, False]","[ñorngo, ñoro , (ngo) , D [ɓorke, guño, gaay..."
12562,12562.0,root,"JANNG-, JAŊNG-",[True],"[JANNG-, JAŊNG-]"
28010,28010.0,lemma,tuufirde pical,"[True, False, False]","[tuufirde pical , v.av+ext , Z]"
25170,25170.0,lemma,soɓude,"[True, False, False]","[soɓude , v.av , Z]"


In [6]:
co_declared = anyEntityFrame[anyEntityFrame['is'] != 'lemma'].merge(lemmaFrame,how='outer',left_index=True,right_index=True,suffixes=('_root','_lemma')).dropna()
co_declared

,paragraphIndex_root,is_root,text_root,mask_root,run_text_list_root,paragraphIndex_lemma,is_lemma,text_lemma,mask_lemma,run_text_list_lemma
index,,,,,,,,,,
84,84.0,root,AAN,"[True, False, False]","[AAN , aan , prn,ind V. a]",84.0,lemma,aan,"[False, True, False]","[AAN , aan , prn,ind V. a]"
155,155.0,root,ABBERE,"[True, False, False, False]","[ABBERE , abbere , var.- wabbere , DcZ C<F...",155.0,lemma,abbere,"[False, True, False, False]","[ABBERE , abbere , var.- wabbere , DcZ C<F..."
198,198.0,root,ADDUDE,"[True, False, False, False]","[ADDUDE , addude , var.- waddude , DCZ C<F...",198.0,lemma,addude,"[False, True, False, False]","[ADDUDE , addude , var.- waddude , DCZ C<F..."
211,211.0,root,AƊA,"[True, False, False]","[AƊA , aɗa , prn,sbj,lf V. a]",211.0,lemma,aɗa,"[False, True, False]","[AƊA , aɗa , prn,sbj,lf V. a]"
225,225.0,root,AJ-,"[True, False, False, False]","[AJ- , ajaade , var.- aajaade , D]",225.0,lemma,ajaade,"[False, True, False, False]","[AJ- , ajaade , var.- aajaade , D]"
...,...,...,...,...,...,...,...,...,...,...
32309,32309.0,root,ƳI'AL,"[True, False, False, False]","[ƳI'AL , ƴi'al , var.- gi'al , H]",32309.0,lemma,ƴi'al,"[False, True, False, False]","[ƳI'AL , ƴi'al , var.- gi'al , H]"
32342,32342.0,root,ƳIWDE,"[True, False, False, False]","[ƳIWDE , ƴiwde , var.- iwde; V. iw- , C <M...",32342.0,lemma,ƴiwde,"[False, True, False, False]","[ƳIWDE , ƴiwde , var.- iwde; V. iw- , C <M..."
32343,32343.0,root,ƳIYAL,"[True, False, False, False]","[ƳIYAL , ƴiyal , var.- gi'al , C <G,M>]",32343.0,lemma,ƴiyal,"[False, True, False, False]","[ƳIYAL , ƴiyal , var.- gi'al , C <G,M>]"


In [7]:
def enumerate_pandas_list(df,target_col,new_col):
   df[new_col] = df[target_col].apply(lambda x: [i[0] for i in enumerate(list(x))] if x is not np.nan else np.nan)
   return df

lemmaFrame_enumer = enumerate_pandas_list(lemmaFrame,'mask','lemma_run_numbers')
lemmaFrame_enumer = lemmaFrame_enumer.explode(['lemma_run_numbers','mask','run_text_list'])
cond1 = lemmaFrame_enumer.loc[:,'lemma_run_numbers'] == 0
cond2 = lemmaFrame_enumer.loc[:,'mask'] == False
false_lemmas = lemmaFrame_enumer.loc[cond1 & cond2,:]
false_lemmas = false_lemmas[~false_lemmas.index.isin(co_declared.index)]

print(anyEntityFrame.shape)
anyEntityFrame = anyEntityFrame[~anyEntityFrame.index.isin(false_lemmas.index)]
print(anyEntityFrame.shape)
print(false_lemmas.shape)
print(nonentityParaFrame.shape)
nonentityParaFrame = nonentityParaFrame.combine_first(false_lemmas[['paragraphIndex']])
print(nonentityParaFrame.shape)


(15266, 5)
(15226, 5)
(40, 6)
(17234, 1)
(17274, 1)


In [8]:
rootFrame_enumer = enumerate_pandas_list(rootFrame,'mask','root_run_numbers')
rootFrame_enumer = rootFrame_enumer.explode(['root_run_numbers','mask','run_text_list'])
cond1 = rootFrame_enumer.loc[:,'root_run_numbers'] == 0
cond2 = rootFrame_enumer.loc[:,'mask'] == False
false_roots = rootFrame_enumer.loc[cond1 & cond2,:]
false_roots = false_roots[~false_roots.index.isin(co_declared.index)]

# print(anyEntityFrame.shape)
# anyEntityFrame = anyEntityFrame[~anyEntityFrame.index.isin(false_roots.index)]
# print(anyEntityFrame.shape)
print(false_roots.shape)
# print(nonentityParaFrame.shape)
# nonentityParaFrame = nonentityParaFrame.combine_first(false_roots[['paragraphIndex']])
# print(nonentityParaFrame.shape)


(0, 6)


In [9]:
rootsubpieceFrame_enumer = enumerate_pandas_list(rootsubpieceFrame,'mask','rootsubpieceFrame_run_numbers')
rootsubpieceFrame_enumer = rootsubpieceFrame_enumer.explode(['rootsubpieceFrame_run_numbers','mask','run_text_list'])
cond1 = rootsubpieceFrame_enumer.loc[:,'rootsubpieceFrame_run_numbers'] == 0
cond2 = rootsubpieceFrame_enumer.loc[:,'mask'] == False
false_rootsubpieceFrames = rootsubpieceFrame_enumer.loc[cond1 & cond2,:]
false_rootsubpieceFrames = false_rootsubpieceFrames[~false_rootsubpieceFrames.index.isin(co_declared.index)]

# print(anyEntityFrame.shape)
# anyEntityFrame = anyEntityFrame[~anyEntityFrame.index.isin(false_rootsubpieceFrames.index)]
# print(anyEntityFrame.shape)
print(false_rootsubpieceFrames.shape)
# print(nonentityParaFrame.shape)
# nonentityParaFrame = nonentityParaFrame.combine_first(false_rootsubpieceFrames[['paragraphIndex']])
# print(nonentityParaFrame.shape)

(0, 6)


## Creating Inheritance / Hierarchy

Now these frames are set up, we can begin the routine. 

At each step, 
   >we merge the possible parents frame with the specific child frame.
   >then we forward-fill the parent portion of the frame to indicate the headship for every record.
   >- we want to 'unnest' this datastructure, and so we need to move from 'dictionary' style to 'record' style.
   >- records require all pertinent infromation be present inside the record (records are usually defined as a single row in most industries)
   >- so forward filling all possible parents will allow 'parentage' be infered by position / precedence on the page.
   >  - a lemma is the child of the nearest 'parent' root UP the page from it, regardless if a root is directly below it.
   >     - therefor forward-fill will allow these inferred relations be directly documented in each record, each row. 
   >     - SQL design says don't repeat yourself, but here we WANT to duplicate the parent value onto each child.
   >     - This allows us to safely remove a 'child' from its context on the page without loosing who it's parents are.
   >- then we merge in the 'frame' of next parent group, then child group, and repeat the forward filling 
   > 
   > *the process is iterative/recursive.*

However, the parent/child relationships happen to align such that each next lower group of parents is composed of the parents + the child from the previous iteration...
As a result we actually have a much simpler structure: we only need to merge each parent grouping directly, and forward fill all of those

We will have some extra duplication from top parents being copied down on each level between them and their children, but we will solve this after we create it.

The extra reason to group it this way is it does allow for a root to be declared on the same paragraph/line as a lemma, which is fairly common from manual review of the dictionary word document

In [10]:
print(co_declared.columns)
print(any_roots.columns)

Index(['paragraphIndex_root', 'is_root', 'text_root', 'mask_root',
       'run_text_list_root', 'paragraphIndex_lemma', 'is_lemma', 'text_lemma',
       'mask_lemma', 'run_text_list_lemma'],
      dtype='object')
Index(['paragraphIndex', 'is', 'text', 'mask', 'run_text_list'], dtype='object')


In [40]:
#subroot-parent columns -> merged with <- lemma-parents columns
entityInheritanceFrame = root_parent.merge(any_roots, on='index',how ='outer',suffixes=['_main_root',''],sort=True, validate='one_to_one')

#'subroot-parent' and 'lemma-parent' columns -> merged with <- 'paragraph-parents' columns

entityInheritanceFrame = entityInheritanceFrame.merge(anyEntityFrame, on='index',how ='outer',suffixes=['_any_root','_any_entity'],sort=True, validate='one_to_one')

# beforedf = entityInheritanceFrame.copy(deep=True)
co_declared_lemmas = anyEntityFrame[anyEntityFrame['is'] != 'lemma'].merge(lemmaFrame,how='outer',left_index=True,right_index=True,suffixes=('_drop','_any_entity')).dropna()
keep_cols = [x for x in co_declared_lemmas.columns if x[-5:]!='_drop']
entityInheritanceFrame = co_declared_lemmas[keep_cols].combine_first(entityInheritanceFrame)
#before forward-fill, but we want to remove "non-paragraph" rows from our data (if present), since we don't care about empty paragraphs as children to our 'parents' (remembering that empty paragraphs are all np.nan)
#this should have no np.nan values in the final paragraph-parents column, but lets check before we accidentally hide the evidence
# print(entityInheritanceFrame.shape) #>>> (15226, 12)
entityInheritanceFrame = entityInheritanceFrame.dropna(subset=['paragraphIndex_any_entity'])
# print(entityInheritanceFrame.shape) #>>> (15226, 12)

#forward-fill np.nan values in the whole dataframe
entityInheritanceFrame = entityInheritanceFrame.fillna(method = 'ffill') #this can be skipped here, as it is redundant, but its a valuable midway point, and informative to review and check our assumptions

#'subroot-parent' and 'lemma-parent' and 'paragraph-parents' columns -> merged with <- 'paragraphs' columns
#note the 'paragraphs' paraFrame contains valid and invalid paragraphs, like the cleaner-output frame we dealt with above.
#it specifically only contains the indexes of paragraphs that are valid AND did not contain on of the entities we've been dealing with (roots, subroots, lemmas)
nan_free_paraFrame = nonentityParaFrame.dropna() #this will remove all non-paragraph records
allParagraphsInheritanceFrame = entityInheritanceFrame.merge(nan_free_paraFrame, on='index',how ='outer',suffixes=['','_paragraphs'],sort=True, validate='one_to_one')                                       
#since we dropped np.nan values above, the only nan values should be introduced by the paragraph merge. Where present in the paragraph columns, these will be the declaration columns of the lemmas and roots.
#to solve that we will forwardfill all columns BUT the new paragraphIndex column, which will fill in the parents for the new paragraphs.
allParagraphsInheritanceFrame.loc[:,entityInheritanceFrame.columns] = allParagraphsInheritanceFrame.loc[:,entityInheritanceFrame.columns].fillna(method = 'ffill')

#here, we can choose to dropna in the dataframe, which will remove all records that do not have a child paragraph in them.
#this will be necessary for creating records to match to the wordnet, since the paragraphs will be our glosses we need.
#however, any record that has a lemma, but does not have paragraphs still has valuable information, so we don't want to discard these completely, we just can't use them directly to search the wordnet.

# identify_condition = partial(lambda x: bool(re.match('is_',x)))
# subroot_condition = partial(lambda x: bool(re.search('_root_subpiece',x)) or bool(re.search('_anyroot',x)))

# print(entityInheritanceFrame.shape) #(15226, 15)
allParagraphsInheritanceFrame  # 32033 rows × 16 columns
#TODO break oneliners out, break roots out of those, check numbers with other checks
# beforedf

,is_any_entity,is_any_root,is_main_root,lemma_run_numbers,mask_any_entity,mask_any_root,mask_main_root,paragraphIndex_any_entity,paragraphIndex_any_root,paragraphIndex_main_root,run_text_list_any_entity,run_text_list_any_root,run_text_list_main_root,text_any_entity,text_any_root,text_main_root,paragraphIndex
index,,,,,,,,,,,,,,,,,
4,root,root,root,NaN,[True],[True],[True],4.0,4.0,4.0,[A],[A],[A],A,A,A,NaN
5,lemma,root,root,NaN,"[True, False, False]",[True],[True],5.0,4.0,4.0,"[a , prn,sbj,sf , DFZH Z<->]",[A],[A],a,A,A,NaN
6,lemma,root,root,NaN,"[True, False, False]",[True],[True],5.0,4.0,4.0,"[a , prn,sbj,sf , DFZH Z<->]",[A],[A],a,A,A,6.0
7,lemma,root,root,NaN,"[True, False, False]",[True],[True],5.0,4.0,4.0,"[a , prn,sbj,sf , DFZH Z<->]",[A],[A],a,A,A,7.0
8,lemma,root,root,NaN,"[True, False, False]",[True],[True],8.0,4.0,4.0,"[-a , suf,pos , F]",[A],[A],-a,A,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32502,lemma,root,root,"[0, 1, 2]","[True, False, False]",[True],[True],32500.0,32499.0,32499.0,"[ƴuuƴude , v.av , D]",[ƳUUƳ-],[ƳUUƳ-],ƴuuƴude,ƳUUƳ-,ƳUUƳ-,32502.0
32503,root,root,root,"[0, 1, 2]",[True],[True],[True],32503.0,32503.0,32503.0,[ƳUUƳ- (2?)],[ƳUUƳ- (2?)],[ƳUUƳ- (2?)],ƳUUƳ- (2?),ƳUUƳ- (2?),ƳUUƳ- (2?),NaN
32504,lemma,root,root,"[0, 1, 2]","[True, False, False]",[True],[True],32504.0,32503.0,32503.0,"[ƴuuƴude , v.av , D]",[ƳUUƳ- (2?)],[ƳUUƳ- (2?)],ƴuuƴude,ƳUUƳ- (2?),ƳUUƳ- (2?),NaN


In [41]:
# afterdf.sample(40)

In [42]:
allParagraphsInheritanceFrame[allParagraphsInheritanceFrame.index.isin(co_declared.index)]

,is_any_entity,is_any_root,is_main_root,lemma_run_numbers,mask_any_entity,mask_any_root,mask_main_root,paragraphIndex_any_entity,paragraphIndex_any_root,paragraphIndex_main_root,run_text_list_any_entity,run_text_list_any_root,run_text_list_main_root,text_any_entity,text_any_root,text_main_root,paragraphIndex
index,,,,,,,,,,,,,,,,,
84,lemma,root,root,"[0, 1, 2]","[False, True, False]","[True, False, False]","[True, False, False]",84.0,84.0,84.0,"[AAN , aan , prn,ind V. a]","[AAN , aan , prn,ind V. a]","[AAN , aan , prn,ind V. a]",aan,AAN,AAN,NaN
155,lemma,root,root,"[0, 1, 2, 3]","[False, True, False, False]","[True, False, False, False]","[True, False, False, False]",155.0,155.0,155.0,"[ABBERE , abbere , var.- wabbere , DcZ C<F...","[ABBERE , abbere , var.- wabbere , DcZ C<F...","[ABBERE , abbere , var.- wabbere , DcZ C<F...",abbere,ABBERE,ABBERE,NaN
198,lemma,root,root,"[0, 1, 2, 3]","[False, True, False, False]","[True, False, False, False]","[True, False, False, False]",198.0,198.0,198.0,"[ADDUDE , addude , var.- waddude , DCZ C<F...","[ADDUDE , addude , var.- waddude , DCZ C<F...","[ADDUDE , addude , var.- waddude , DCZ C<F...",addude,ADDUDE,ADDUDE,NaN
211,lemma,root,root,"[0, 1, 2]","[False, True, False]","[True, False, False]","[True, False, False]",211.0,211.0,211.0,"[AƊA , aɗa , prn,sbj,lf V. a]","[AƊA , aɗa , prn,sbj,lf V. a]","[AƊA , aɗa , prn,sbj,lf V. a]",aɗa,AƊA,AƊA,NaN
225,lemma,root,root,"[0, 1, 2, 3]","[False, True, False, False]","[True, False, False, False]","[True, False, False, False]",225.0,225.0,225.0,"[AJ- , ajaade , var.- aajaade , D]","[AJ- , ajaade , var.- aajaade , D]","[AJ- , ajaade , var.- aajaade , D]",ajaade,AJ-,AJ-,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32309,lemma,root,root,"[0, 1, 2, 3]","[False, True, False, False]","[True, False, False, False]","[True, False, False, False]",32309.0,32309.0,32309.0,"[ƳI'AL , ƴi'al , var.- gi'al , H]","[ƳI'AL , ƴi'al , var.- gi'al , H]","[ƳI'AL , ƴi'al , var.- gi'al , H]",ƴi'al,ƳI'AL,ƳI'AL,NaN
32342,lemma,root,root,"[0, 1, 2, 3]","[False, True, False, False]","[True, False, False, False]","[True, False, False, False]",32342.0,32342.0,32342.0,"[ƳIWDE , ƴiwde , var.- iwde; V. iw- , C <M...","[ƳIWDE , ƴiwde , var.- iwde; V. iw- , C <M...","[ƳIWDE , ƴiwde , var.- iwde; V. iw- , C <M...",ƴiwde,ƳIWDE,ƳIWDE,NaN
32343,lemma,root,root,"[0, 1, 2, 3]","[False, True, False, False]","[True, False, False, False]","[True, False, False, False]",32343.0,32343.0,32343.0,"[ƳIYAL , ƴiyal , var.- gi'al , C <G,M>]","[ƳIYAL , ƴiyal , var.- gi'al , C <G,M>]","[ƳIYAL , ƴiyal , var.- gi'al , C <G,M>]",ƴiyal,ƳIYAL,ƳIYAL,NaN


In [43]:
entityDeclarationParas = allParagraphsInheritanceFrame[allParagraphsInheritanceFrame['paragraphIndex'].isna()]
entityDeclarationParasLEN = entityDeclarationParas.shape[0]
print('length of entity declaration Frame: ',entityDeclarationParasLEN)
summedEntityFramesLEN = rootFrame.shape[0]+rootsubpieceFrame.shape[0]+lemmaFrame.shape[0]
print('summed length of all 3 entity frames: ',summedEntityFramesLEN)
print('difference between the two: ',summedEntityFramesLEN-entityDeclarationParasLEN)
print('number of misclassified lemmas that were corrected: ',false_lemmas.shape[0])
print('intersection of root-lemma frames: ',len(rootFrame.index.intersection(lemmaFrame.index)))
# >>>intersection of root-lemma frames:  1438
print('intersection of subroot-lemma frames: ',len(rootsubpieceFrame.index.intersection(lemmaFrame.index)))
# >>>intersection of subroot-lemma frames:  0
print('intersection of root-subroot frames: ',len(rootFrame.index.intersection(rootsubpieceFrame.index)))
# >>>intersection of root-subroot frames:  0
paragraphRecordsFrame = allParagraphsInheritanceFrame[~allParagraphsInheritanceFrame['paragraphIndex'].isna()]
print('number of content (non-entity) paragraphs: ',paragraphRecordsFrame.shape[0])
#Conclusion: entityDeclarationParas is correct from our intuition.

length of entity declaration Frame:  15226
summed length of all 3 entity frames:  16704
difference between the two:  1478
number of misclassified lemmas that were corrected:  40
intersection of root-lemma frames:  1438
intersection of subroot-lemma frames:  0
intersection of root-subroot frames:  0
number of content (non-entity) paragraphs:  16807


In [44]:
print('unique parent types for paragraphs: ',paragraphRecordsFrame['is_any_entity'].unique(),'  ...  total found: ',len(paragraphRecordsFrame['is_any_entity'].unique()),'\n')
print(paragraphRecordsFrame.groupby('is_any_entity')['paragraphIndex'].count())

unique parent types for paragraphs:  ['lemma']   ...  total found:  1 

is_any_entity
lemma    16807
Name: paragraphIndex, dtype: int64


In [46]:
# paragraphRecordsFrame

## Cleaning up column names and restructuring the nature of subroot and lemma columns since data is now clean

In [47]:
paragraphRecordsFrame.columns

order_for_cols = [
   'paragraphIndex_main_root', 
   'is_main_root', 
   'text_main_root',
   'mask_main_root', 
   'run_text_list_main_root', 
   'paragraphIndex_any_root',
   'is_any_root', 
   'text_any_root', 
   'mask_any_root',
   'run_text_list_any_root', 
   'paragraphIndex_any_entity', 
   'is_any_entity',
   'text_any_entity', 
   'mask_any_entity', 
   'run_text_list_any_entity',
   'paragraphIndex',
   ]
assert(not bool(set(order_for_cols).difference(set(paragraphRecordsFrame.columns))))
paragraphRecordsFrame = paragraphRecordsFrame[order_for_cols]

expunge_root_from_subroot = [   
   'paragraphIndex_any_root',
   'is_any_root', 
   'text_any_root', 
   'mask_any_root',
   'run_text_list_any_root', 
   # 'paragraphIndex_any_entity', 
   # 'is_any_entity',
   # 'text_any_entity', 
   # 'mask_any_entity', 
   # 'run_text_list_any_entity',
   ]
cols_mapper = {
   'paragraphIndex_main_root':'paragraphIndex_root', 
   'is_main_root':'is_root', 
   'text_main_root':'text_root',
   'mask_main_root':'mask_root', 
   'run_text_list_main_root':'run_text_list_root', 
   'paragraphIndex_any_root':'paragraphIndex_root_subpiece',
   'is_any_root':'is_root_subpiece', 
   'text_any_root':'text_root_subpiece', 
   'mask_any_root':'mask_root_subpiece',
   'run_text_list_any_root':'run_text_list_root_subpiece', 
   'paragraphIndex_any_entity':'paragraphIndex_lemma', 
   'is_any_entity':'is_lemma',
   'text_any_entity':'text_lemma', 
   'mask_any_entity':'mask_lemma', 
   'run_text_list_any_entity':'run_text_list_lemma',
   }
drop_cols = [
   # 'paragraphIndex_main_root', 
   'is_main_root', 
   'text_main_root',
   'mask_main_root', 
   # 'run_text_list_main_root', 
   # 'paragraphIndex_any_root',
   'is_any_root', 
   'text_any_root', 
   'mask_any_root',
   # 'run_text_list_any_root', 
   # 'paragraphIndex_any_entity', 
   'is_any_entity',
   # 'text_any_entity', 
   # 'mask_any_entity', 
   # 'run_text_list_any_entity',
   # 'paragraphIndex'
   ]

simple_col_list = [
   'paragraphIndex_root', 
   # 'is_main_root', 
   # 'text_main_root',
   'mask_main_root', 
   'run_text_list_root', 
   'paragraphIndex_root_subpiece',
   # 'is_any_root', 
   # 'text_any_root', 
   'mask_any_root',
   'run_text_list_root_subpiece', 
   'paragraphIndex_lemma', 
   # 'is_any_entity',
   'text_lemma', 
   'mask_lemma', 
   'run_text_list_lemma',
   'paragraphIndex_paragraph',
   'run_text_list_paragraph'
   ]
paragraphRecordsFrame['run_text_list_paragraph'] = paragraphRecordsFrame['paragraphIndex'].apply(lambda x: parsed_object_lookup[x].run_text)
# print(paragraphRecordsFrame.columns)
paragraphRecordsFrame = paragraphRecordsFrame.copy(deep=True)
paragraphRecordsFrame.loc[paragraphRecordsFrame['is_any_root']=='root',expunge_root_from_subroot] = np.nan
# print(paragraphRecordsFrame.columns)
paragraphRecordsFrame.rename(mapper=cols_mapper,axis = 1,inplace=True)
# print(paragraphRecordsFrame.columns)
# test_frame = paragraphRecordsFrame.copy(deep=True)
# test_frame.drop(drop_cols, axis = 1, inplace=True)
# test_frame.columns = colsnewnames
# test_frame
masks = ['mask_root','mask_root_subpiece','mask_lemma']
for m in masks:
   paragraphRecordsFrame['run_enumerated'+m[4:]] = paragraphRecordsFrame[m].apply(lambda x: [i[0] for i in enumerate(list(x))] if x is not np.nan else np.nan)
paragraphRecordsFrame
# assert(not bool(set(order_for_cols).difference(set(paragraphRecordsFrame.columns))))
# print(len(paragraphRecordsFrame.columns))

,paragraphIndex_root,is_root,text_root,mask_root,run_text_list_root,paragraphIndex_root_subpiece,is_root_subpiece,text_root_subpiece,mask_root_subpiece,run_text_list_root_subpiece,paragraphIndex_lemma,is_lemma,text_lemma,mask_lemma,run_text_list_lemma,paragraphIndex,run_text_list_paragraph,run_enumerated_root,run_enumerated_root_subpiece,run_enumerated_lemma
index,,,,,,,,,,,,,,,,,,,,
6,4.0,root,A,[True],[A],NaN,NaN,NaN,NaN,NaN,5.0,lemma,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",6.0,[you (sg.)],[0],NaN,"[0, 1, 2]"
7,4.0,root,A,[True],[A],NaN,NaN,NaN,NaN,NaN,5.0,lemma,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",7.0,[tu],[0],NaN,"[0, 1, 2]"
9,4.0,root,A,[True],[A],NaN,NaN,NaN,NaN,NaN,8.0,lemma,-a,"[True, False, False]","[-a , suf,pos , F]",9.0,[your (sg.) (only with certain nouns such as t...,[0],NaN,"[0, 1, 2]"
10,4.0,root,A,[True],[A],NaN,NaN,NaN,NaN,NaN,8.0,lemma,-a,"[True, False, False]","[-a , suf,pos , F]",10.0,"[ton, ta, tes (seulement pour certains substan...",[0],NaN,"[0, 1, 2]"
12,4.0,root,A,[True],[A],NaN,NaN,NaN,NaN,NaN,11.0,lemma,aan,"[True, False, False]","[aan , prn,ind , DFZH [an]:Z<->]",12.0,[you (sg.) (emphatic)],[0],NaN,"[0, 1, 2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32498,32495.0,root,ƳUUW-,[True],[ƳUUW-],NaN,NaN,NaN,NaN,NaN,32496.0,lemma,ƴuuwaade,"[True, False, False]","[ƴuuwaade , v.mv , D [ƴuubaade, ƴooraade]]",32498.0,[s'incliner; se cabrer],[0],NaN,"[0, 1, 2]"
32501,32499.0,root,ƳUUƳ-,[True],[ƳUUƳ-],NaN,NaN,NaN,NaN,NaN,32500.0,lemma,ƴuuƴude,"[True, False, False]","[ƴuuƴude , v.av , D]",32501.0,[to increase in volume],[0],NaN,"[0, 1, 2]"
32502,32499.0,root,ƳUUƳ-,[True],[ƳUUƳ-],NaN,NaN,NaN,NaN,NaN,32500.0,lemma,ƴuuƴude,"[True, False, False]","[ƴuuƴude , v.av , D]",32502.0,[augmenter de volume],[0],NaN,"[0, 1, 2]"


### Checking for cleaning issues and augmenting the data slightly for easier processing

In [31]:
# def get_entity_index_from_para_record(source_df:pd.DataFrame, index:pd.Index,entity_index_col_names = ['paragraphIndex_root','paragraphIndex_root_subpiece','paragraphIndex_lemma']):
#    print(source_df.head())
#    return source_df.loc[index,:]
   
# def get_para_text(df:pd.DataFrame,  test_col_name:str,  lookup:Dict[int,Any],  index:bool=False,  column:Optional[str] = None):

#    if column is not None and index is not False:
#       raise ValueError('index and column values cannot both be passed.')
#    if index is not False:
#       df.loc[:,test_col_name] = df.index.values
#    if column is not None:
#       df.loc[:,test_col_name] = df[column]
#    df.loc[:,test_col_name] = df.loc[:,test_col_name].apply(lambda x: lookup[x].run_text)
#    return df
# def horizontal_explode


In [48]:
pd.set_option('display.max_rows', 200)
test_frame = paragraphRecordsFrame.copy(deep=True)
print(parsed_object_lookup[24476])
# print(paragraphRecordsFrame.loc[24474,:])
lemma_mask_frame = test_frame['mask_lemma'].apply(pd.Series)
lemma_mask_frame
# print(lemma_mask_frame[lemma_mask_frame[0]==False])
test_frame.loc[24476]
bold_where_not_expected = lemma_mask_frame[lemma_mask_frame[[1,2,3,4]].isin([True]).any(axis=1)] #94, only 2 dont

bold_out_of_place = lemma_mask_frame[lemma_mask_frame[0]!=True] #only 2 arent in rows with lemmas 24476, 29842

test_frame_lem_exploded = test_frame.loc[bold_where_not_expected.index,:].explode(['run_enumerated_lemma','mask_lemma','run_text_list_lemma'])
# get_entity_index_from_para_record(source_df = test_frame, index = bold_where_not_expected.index)
# bold_where_not_expected = get_para_text(df = bold_where_not_expected, test_col_name = 'paragraph_run_text', lookup = parsed_object_lookup,column='paragraphIndex_lemma')
# bold_where_not_expected
# test_frame_lem_exploded.loc[test_frame_lem_exploded.run_enumerated_lemma.values == 0 &  test_frame_lem_exploded.mask_lemma.values == False,:]
run_is_zero = test_frame_lem_exploded.run_enumerated_lemma == 0
mask_is_false = test_frame_lem_exploded.mask_lemma.values == False
test_frame_lem_exploded.loc[run_is_zero &  mask_is_false,:]



run_italic=[None] run_bold=[None] para_first_line_indent=None run_font_name=['TmsRmn 8pt'] para_text='ceertugol (Math.)(T): disjunction ; disjonction' paragraph_enumeration=24476 run_font_size_pt=[8.0] run_text=['ceertugol (Math.)(T): disjunction ; disjonction'] para_left_indent=116128800


,paragraphIndex_root,is_root,text_root,mask_root,run_text_list_root,paragraphIndex_root_subpiece,is_root_subpiece,text_root_subpiece,mask_root_subpiece,run_text_list_root_subpiece,paragraphIndex_lemma,is_lemma,text_lemma,mask_lemma,run_text_list_lemma,paragraphIndex,run_text_list_paragraph,run_enumerated_root,run_enumerated_root_subpiece,run_enumerated_lemma
index,,,,,,,,,,,,,,,,,,,,


In [51]:
allentitymasks = entityDeclarationParas['mask_any_entity'].apply(pd.Series)

In [52]:
entity_out_of_place = allentitymasks[allentitymasks[0]!=True]

In [53]:
entity_out_of_place.loc[:,'paragraph_run_text'] = entity_out_of_place.index.values
entity_out_of_place.loc[:,'paragraph_run_text'] = entity_out_of_place.loc[:,'paragraph_run_text'].apply(lambda x: parsed_object_lookup[x].run_text)
entity_out_of_place

c:\Users\peter\VSC-Workspace_PULAR\.pular_venv\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
c:\Users\peter\VSC-Workspace_PULAR\.pular_venv\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,0,1,2,3,4,5,6,7,paragraph_run_text
index,,,,,,,,,
84,False,True,False,NaN,NaN,NaN,NaN,NaN,"[AAN , aan , prn,ind V. a]"
155,False,True,False,False,NaN,NaN,NaN,NaN,"[ABBERE , abbere , var.- wabbere , DcZ C<F..."
198,False,True,False,False,NaN,NaN,NaN,NaN,"[ADDUDE , addude , var.- waddude , DCZ C<F..."
211,False,True,False,NaN,NaN,NaN,NaN,NaN,"[AƊA , aɗa , prn,sbj,lf V. a]"
225,False,True,False,False,NaN,NaN,NaN,NaN,"[AJ- , ajaade , var.- aajaade , D]"
...,...,...,...,...,...,...,...,...,...
32309,False,True,False,False,NaN,NaN,NaN,NaN,"[ƳI'AL , ƴi'al , var.- gi'al , H]"
32342,False,True,False,False,NaN,NaN,NaN,NaN,"[ƳIWDE , ƴiwde , var.- iwde; V. iw- , C <M..."
32343,False,True,False,False,NaN,NaN,NaN,NaN,"[ƳIYAL , ƴiyal , var.- gi'al , C <G,M>]"


In [49]:
lemma_dec_paras = entityDeclarationParas[entityDeclarationParas['is_any_entity']=='lemma']
print('total lemmas in our data: ',lemma_dec_paras.shape[0])
# lemma_dec_paras

total lemmas in our data:  9570


In [50]:
print('number of lemmas with some paragraphs: ',len(paragraphRecordsFrame.groupby('paragraphIndex_lemma')))

number of lemmas with some paragraphs:  7994


In [57]:
# all_inheritance_frames_dict = {
#    'allParagraphsInheritanceFrame':allParagraphsInheritanceFrame,
#    'entityDeclarationParas':entityDeclarationParas,
#    'paragraphRecordsFrame':paragraphRecordsFrame
# }
all_inheritance_frames_tup = (
   allParagraphsInheritanceFrame,
   entityDeclarationParas,
   paragraphRecordsFrame
)
with open('pickled_results/all_inheritance_frames_tup.pkl', 'wb') as file:
   pickle.dump(all_inheritance_frames_tup, file)

In [59]:
# entity_out_of_place_run_text_exploded = entity_out_of_place['paragraph_run_text'].apply(pd.Series)
# entity_out_of_place_run_text_exploded
allParagraphsInheritanceFrame

,is_any_entity,is_any_root,is_main_root,lemma_run_numbers,mask_any_entity,mask_any_root,mask_main_root,paragraphIndex_any_entity,paragraphIndex_any_root,paragraphIndex_main_root,run_text_list_any_entity,run_text_list_any_root,run_text_list_main_root,text_any_entity,text_any_root,text_main_root,paragraphIndex
index,,,,,,,,,,,,,,,,,
4,root,root,root,NaN,[True],[True],[True],4.0,4.0,4.0,[A],[A],[A],A,A,A,NaN
5,lemma,root,root,NaN,"[True, False, False]",[True],[True],5.0,4.0,4.0,"[a , prn,sbj,sf , DFZH Z<->]",[A],[A],a,A,A,NaN
6,lemma,root,root,NaN,"[True, False, False]",[True],[True],5.0,4.0,4.0,"[a , prn,sbj,sf , DFZH Z<->]",[A],[A],a,A,A,6.0
7,lemma,root,root,NaN,"[True, False, False]",[True],[True],5.0,4.0,4.0,"[a , prn,sbj,sf , DFZH Z<->]",[A],[A],a,A,A,7.0
8,lemma,root,root,NaN,"[True, False, False]",[True],[True],8.0,4.0,4.0,"[-a , suf,pos , F]",[A],[A],-a,A,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32502,lemma,root,root,"[0, 1, 2]","[True, False, False]",[True],[True],32500.0,32499.0,32499.0,"[ƴuuƴude , v.av , D]",[ƳUUƳ-],[ƳUUƳ-],ƴuuƴude,ƳUUƳ-,ƳUUƳ-,32502.0
32503,root,root,root,"[0, 1, 2]",[True],[True],[True],32503.0,32503.0,32503.0,[ƳUUƳ- (2?)],[ƳUUƳ- (2?)],[ƳUUƳ- (2?)],ƳUUƳ- (2?),ƳUUƳ- (2?),ƳUUƳ- (2?),NaN
32504,lemma,root,root,"[0, 1, 2]","[True, False, False]",[True],[True],32504.0,32503.0,32503.0,"[ƴuuƴude , v.av , D]",[ƳUUƳ- (2?)],[ƳUUƳ- (2?)],ƴuuƴude,ƳUUƳ- (2?),ƳUUƳ- (2?),NaN


In [ ]:
# next steps, force entities to have a starting run with the value. "match" for the feature, not search
#repeat cleaning, groupby lemma index, begin work on dictionary extraction with what I have. Record metrics of the values being filtered.
# some ongoing attrition score

In [55]:
pd.reset_option("^display")

In [ ]:
outputs = [
   
   'txt_FulaLemmas',
   'txt_FulaRoots',
   
   #extra
   'txt_FulaAnnotations',
   #gloss derivatives
   'txt_FulaSenseEnglish',
   'txt_FulaSenseFrench',
   'txt_FulaSenseClassifications',
   'txt_FulaSenseEnglishAnnotations',
   'txt_FulaSenseFrenchAnnotations',

   #lemma derivative
   'txt_FulaDialects',
   'txt_FulaPOSTags',
   'txt_FulaSynonyms',


   #agg
   'txt_FulaInParenthesis',
   'txt_POS',
   'txt_RootOrigins'
]

In [ ]:
# noun_patterns = [r'n\.',r"(?<=\()[^\)]+",r"\/"]
# nounPatternRegex = re.compile('|'.join([p for p in noun_patterns]))
# print(nounPatternRegex)
# s = 'n. dfhjkgqh (wsh) askldjhf / asdfhjkkh'
# print(re.findall(nounPatternRegex,s))
# print(nounPatternRegex.findall(s))

# if /, the next word is a plural. The plural also then usually has a (el) class after it. commas are multiple version
# so each lemma noun may have a (class). The lemma may have one or multiple plurals, which will have a modified (class)
# so have a nouns list? nouns may then be 
# [('n.',{bool}), #only needed if this gets added to all POS? Or maybe theres information in its use?
# ([[{word}],{Optional[class]}]),  #singular
   #([[{word}],{Optional[class]}])]   #plural

# so nounStruct[0][1] is the flag for noun?
   # one POS table, and noun will get added there. But the presence of a noun indicates to look here
# classes may be found [sClass[1] for sClass in nounStruct[1]] will give all classes of singular, [sClass[1] for sClass in nounStruct[1]]

In [ ]:
# now = datetime.now()
# current_time = now.strftime("%Y-%m-%d_-_%H-%M-%S")
# print(f"Experiment time: {current_time}\nExperiment note: {experiment}\n\n")

In [ ]:
def getAssert(dct,val,typ:type, strict = True) -> Any:
   out = dct.get(val,None)
   if strict and typ is type(None):
      raise NotImplementedError(f'this get/assert function cannot "get" None values safely, and a {val} type was passed for dict {dict}')
   if strict and out is None:
      raise KeyError(f"the provided dict {dct} did not have the key: {val}")
   assert isinstance(out,typ), f"the provided dict {dct} did not give {val} with the expected type: {typ}, but instead {out}"
   return out
   
def trustyGet(obj:Docx_Paragraph_and_Runs, feat: str, silent_return = True) -> str:
      if not silent_return:
         output : str = getattr(obj,feat,'')
         if len(output) > 0:
            return output
         else:
            raise AttributeError(f'trusty getter could not find: {feat}')
      else:
         output : str = getattr(obj,feat,'')
         return output

In [ ]:
class dict_entry(pydantic.BaseModel): 
   '''
   #True is used as default value, and must be changed to False if the feature is not found. This will catch not implemented, and incomplete updates
   '''
   irregularities: List[str] = []
   lemma_index: int 
   paragraphs: List[Docx_Paragraph_and_Runs] = Field(...,min_items = 1)
   lemma: str = Field(...,min_length = 1) #article
   root: str
   root_subpiece: Union[str,bool] = True #these will not be updated after initialization
   root_metadata: Dict[str,Any]  #index,root_text, root_meta_data #article 
   root_origin: Union[str,bool] = True #these will not be updated after initialization
   #Paragraph Derivatives: Glosses and Annotations
   lemmaLine_runs: List[str] #these will not be updated after initialization
   englishGlossLine_runs: Union[List[str],bool] = True #these will not be updated after initialization
   frenchGlossLine_runs: Union[List[str],bool] = True #these will not be updated after initialization
   FulaAnnotations_runs: Union[List[List[str]],bool]= True #these will not be updated after initialization

   #Gloss Derivatives: English Senses
   FulaSenseEnglish: Union[List[str],bool] = True #list of "senses" split by semicolons #article
   FulaSenseEnglish_Count: Union[int,bool] = True #number of senses in english #aka FulaSenseClassifications #article
   FulaSenseEnglish_Annotations: Union[List[str],bool] = True #contains the annotations (in parenthesis) for a given sense, if any #article 
   FulaSenseEnglish_Synonyms: Union[List[str],bool] = True #holds bracket text for a sense, suspected all synonyms. These may all occur at the end, and may be redundant with the synonyms provided at the head of the entry
   FulaSenseEnglish_unusedContent: Union[List[str],bool] = True #holds any run text that is not contained in the above features
   
   #Gloss Derivatives: French Senses
   FulaSenseFrench: Union[List[str],bool] = True #article
   FulaSenseFrench_Count: Union[int,bool] = True #aka FulaSenseClassifications
   FulaSenseFrench_Annotations: Union[List[str],bool] = True
   FulaSenseFrench_Synonyms: Union[List[str],bool] = True
   FulaSenseFrench_unusedContent: Union[List[str],bool] = True
   
   #lemma derivative
   lemmaLine_unusedContent: List[Tuple[int,str]] #= Field(...) #int is index for run to allow tracing as entries are removed
   FulaDialects: Union[List[str],bool] = True #article
   FulaPOSTags: Union[List[str],bool] = True #article
   FulaPOSClass: Union[List[str],bool] = True #Noun, Adj, Verb, Adv, Prn, ..., Complicated, Indeterminate
   FulaNoun_NounsAndClass: Union[List[Tuple[str,str]],bool] = True #Pular has unique noun classes. Lemma will be copied here beside its class ("noun", "nounclass"), and any additional singular forms will be in additional tuples in this same list
   FulaNoun_PluralsAndClass: Union[List[Tuple[str,str]],bool] = True #Dict provides plurals for nouns. Tuples will have ("noun", "nounclass")
   FulaSynonyms: Union[List[str],bool] = True #article
   FulaCrossRef: Union[List[str],bool] = True
         # FulaVerbClass:
   
   class Config:
      validate_all = True
      validate_assignment = True
      smart_union = True  
      extra = 'forbid'

   @root_validator(pre=True)
   def _validate_and_build(cls, values: Dict[str,Any]) -> Dict[str, Any]:
      # print(values)
      newValues: Dict[str,Any] = {}
      newValues['irregularities'] = []

      ###PARAGRAPHS CHECK###
      print(values)
      newValues['paragraphs'] = getAssert(values,'paragraphs',list) #these come in as tuples(ind,para)
      #TODO have more rigorous checks on para since so much depends on that even in this validation
      assert isinstance(newValues['paragraphs'][0],tuple), 'currently paras are packaged in a tuple with their original doc index'
      newValues['paragraphs'] = [p for i,p in newValues['paragraphs']]
      if len(newValues['paragraphs']) == 0:
         raise ValueError('given invalid paragraphs of zero length')
      else: #logic for subsidary paragraphs
         lemma_line = trustyGet(newValues['paragraphs'][0],feat = 'run_text', silent_return=False)
         newValues['lemmaLine_runs'] = lemma_line
         if len(newValues['paragraphs']) == 1:
            newValues['englishGlossLine_runs'] = False
            newValues['frenchGlossLine_runs'] = False
         elif len(newValues['paragraphs']) == 2:
            newValues['irregularities'].append('What:ambiguous, Where:paragraphs, Why: only one')
            newValues['englishGlossLine_runs'] = False
            newValues['frenchGlossLine_runs'] = False
         elif len(newValues['paragraphs']) == 3:
            newValues['englishGlossLine_runs'] = newValues['paragraphs'][1].get_run_text()
            newValues['frenchGlossLine_runs'] = newValues['paragraphs'][2].get_run_text()
         else: 
            newValues['englishGlossLine_runs'] = newValues['paragraphs'][1].get_run_text()
            newValues['frenchGlossLine_runs'] = newValues['paragraphs'][2].get_run_text()
            newValues['FulaAnnotations_runs'] = newValues['paragraphs'][3:].get_run_text()
      
      
      try: ###LEMMA CHECK###
         #reading in lemma results from first pass of pydantic parser
         lemma_index, lemma_mask, lemmaLine_runs = getAssert(values,'lemma',tuple)
         #checking for correct structure
         lemma_matched_runs = list(compress(lemmaLine_runs,lemma_mask))
         if len(lemma_matched_runs) > 1:
            newValues['irregularities'].append('What:Unexpected, Where: Lemmas, Why: the merge routines should aggregate adjacent runs with same features. Multiple Lemma runs should not be possible if Bold is contiguous')
         lemma_text = ''.join(chain(lemma_matched_runs)).strip()
            #TODO have better control for expected structure that these runs should be adjacent (and only should be one)
         #saving values. These will have to pass type check of declared attribute types for validation to succeed
         newValues['lemma'] = lemma_text
         newValues['lemma_index'] = lemma_index
         newValues['lemmaLine_runs'] = lemmaLine_runs
         used_run_mask = lemma_mask
      except Exception as e:
         #exception lemma check
         raise RuntimeError('failed lemma check') from e

      try: ###ROOT CHECK###
         #reading in root results from first pass of pydantic parser
         #checking for correct structure
         root_index, root_mask, rootLine_runs = getAssert(values,'root',tuple)
         root_matched_runs = list(compress(rootLine_runs,root_mask))
         if len(root_matched_runs) > 1:
            newValues['irregularities'].append('What:Unexpected, Where: Root, Why: the merge routines should aggregate adjacent runs with same features. Multiple Root runs should not be possible if Fontsize is contiguous and unique')
         root_text = ''.join(chain(root_matched_runs)).strip()
         if root_index == newValues['lemma_index']:
            newValues['irregularities'].append('What:inconsistent, Where: Lemmas and Root, Why:normally root and lemma are on different lines. This has them sharing, which may indicate a lack of other content')
            used_run_mask = [any(run) for run in list(zip(used_run_mask,root_mask))]
            rootLine_runs = False
         newValues['root'] = root_text
         newValues['root_metadata'] = {'root_index': root_index, 'root_runs':rootLine_runs}
      except Exception as e:
         #exception ROOT check
         raise RuntimeError('failed ROOT check') from e

      try: ###ROOT-Subpiece CHECK###
         #iterating in case a subroot is present
         root_index, root_mask, rootLine_runs = getAssert(values,'root',tuple)
         root_matched_runs = list(compress(rootLine_runs,root_mask))
         if len(root_matched_runs) > 1:
            newValues['irregularities'].append('What:Unexpected, Where: Root-Subpiece, Why: the merge routines should aggregate adjacent runs with same features. Multiple Root runs should not be possible if Fontsize is contiguous and unique')
         root_subpiece_text = ''.join(chain(root_matched_runs)).strip()
         if root_index == newValues['lemma_index']:
            newValues['irregularities'].append('What:inconsistent, Where: Lemmas and Root-Subpiece, Why:normally root and lemma are on different lines. This has them sharing, which may indicate a lack of other content')
            used_run_mask = [any(run) for run in list(zip(used_run_mask,root_mask))]
            rootLine_runs = False
         newValues['root_subpiece'] = root_subpiece_text
         newValues['root_metadata'] = {'root_subpiece_index': root_index, 'root_subpiece_runs':rootLine_runs}
      except AssertionError as e:
         newValues['root_subpiece'] = False
      except Exception as e:
         #exception ROOT-Subpiece check
         raise RuntimeError('failed ROOT-Subpiece check') from e

      #determining lemmaLine_unusedContent
      unused_run_mask = [not r for r in used_run_mask]
      print(list(compress(enumerate(newValues['lemmaLine_runs']),unused_run_mask)))
      newValues['lemmaLine_unusedContent'] = list(compress(enumerate(newValues['lemmaLine_runs']),unused_run_mask))
      
      return newValues

   def parse_senses(self):
      #TODO
      return

   def parse_glossRemainder(self):
      #TODO
      return

   def parse_lemmaLine(self):
      #TODO
      return

   def parse_lemmaLineRemainder(self):
      #TODO
      return

   def get_rootOrigin(self):
      #TODO
      return
 
   def give_entryText(self, joiner = '\t') -> str: #article
      return joiner.join([para.trustyGet('para_text') for para in self.paragraphs])

   
      
            
# dict_entry.parse_obj({'paragraphs':[b for a,b in parsed_object_list[5:8]]})

# incoming = {
#    'paragraphs': List[Docx_Paragraph_and_Runs],
#    'root': List[int,str,List[bool]],
#    'lemma': List[int,str,List[bool]],
#    # 'pos': 
# }

# [
#    ['paras'], #obj
#    ['parsed_paras'], #pydocx
#    ['roots'], #name, fill forwards
#    ['lemmas']  #text, fill forwards
# ]

In [ ]:
# import pickle
# with open('parsed_objectClass_outcomes_dict.pkl', 'rb') as file:
#     # Call load method to deserialze
#     parsed_objectClass_outcomes_dict = pickle.load(file, encoding='utf-8')

# parsed_object_list = parsed_objectClass_outcomes_dict['parsed_object_list'] 
# para_text_lookup = parsed_objectClass_outcomes_dict['para_text_lookup'] 
# root_ind_list = parsed_objectClass_outcomes_dict['root_ind_list'] 
# subroot_ind_list = parsed_objectClass_outcomes_dict['subroot_ind_list'] 
# lemma_ind_list = parsed_objectClass_outcomes_dict['lemma_ind_list'] 
# root_and_lemma_one_line = parsed_objectClass_outcomes_dict['root_and_lemma_one_line'] 
# root_lookup = parsed_objectClass_outcomes_dict['root_lookup'] 
# lemma_lookup = parsed_objectClass_outcomes_dict['lemma_lookup'] 
# # char_counts = parsed_objectClass_outcomes_dict['char_counts']

In [ ]:
# # docx_filename = "pasted_docx page 1.docx"
# docx_filename = "Fula_Dictionary-repaired.docx"
# parsed_to_dict = read_docx(docx_filename)

# outcomes_dict = monolith_root_and_lemma_processor(parsed_to_dict['parsed_object_list'],parsed_to_dict['char_counts'])

# parsed_object_list = outcomes_dict['parsed_object_list'] 
# para_text_lookup = outcomes_dict['para_text_lookup'] 
# root_ind_list = outcomes_dict['root_ind_list'] 
# subroot_ind_list = outcomes_dict['subroot_ind_list'] 
# lemma_ind_list = outcomes_dict['lemma_ind_list'] 
# root_and_lemma_one_line = outcomes_dict['root_and_lemma_one_line'] 
# root_lookup = outcomes_dict['root_lookup'] 
# lemma_lookup = outcomes_dict['lemma_lookup'] 
# char_counts = outcomes_dict['char_counts']

In [ ]:
# #Creating lists of indexes for use in remaining processing

# # # print(len(parsed_object_list)) #32040
# #note this parsed paras are packaged with the original para number in the docx
# #therefore 32507 is the index to use, but len of that IS 32040, since ~500 were empty paragraphs that were not parsed
# parsed_para_indexes = [i for i,p in parsed_object_list]

# lim = 32507
# # rootInds = [x for x in root_ind_list if x < lim]
# rootInds=root_ind_list
# # subRootInds = [x for x in subroot_ind_list if x < lim]
# subRootInds=subroot_ind_list
# # lemmaInds = [x for x in lemma_ind_list if x < lim]
# lemmaInds=lemma_ind_list

# allroots = rootInds.copy()
# allroots.extend(subRootInds)
# allroots = sorted(allroots)
# allEntities = allroots.copy()
# allEntities.extend(lemmaInds)
# allEntities = sorted(set(allEntities))
# print('allEntities: ',len(allEntities))


# # all_paras = [i for i,obj in parsed_object_list if i < lim]
# normal_para = parsed_para_indexes.copy()
# [normal_para.remove(i) for i in allEntities]
# print('normal_para: ',len(normal_para))

# # normal_para = [x for x in parsed_para_indexes if all([x not in rootInds, x not in lemmaInds, x not in subRootInds])]
# # print(normal_para)
# # print(allroots)
# root_aligned_lemmas = list(closest(pairwise(allroots), lemmaInds))
# lemma_aligned_paras = list(closest(pairwise(lemmaInds),normal_para))

In [ ]:
# import pickle
# with open('entity_index_schema.pkl', 'rb') as file:
#     # Call load method to deserialze
#     entity_index_schema = pickle.load(file, encoding='utf-8')

In [ ]:
# for r,v1 in entity_index_schema.items():
#    root = (r,root_lookup[r][1],root_lookup[r][2])
#    if isinstance(v1,dict):
#       for k,v in v1.items():
#          if k in subroot_ind_list:
#             subroot

In [ ]:
# result = []
# for i in parsed_object_list[5:15]:
#    result.append(dict_entry({i))
# print(root_lookup[4])
# print(para_text_lookup[3])
# root_container = [(ind, mask, rootLine_runs) for ind, mask, rootLine_runs in getAssert(values,'root',list)]
shorter_obj_list = parsed_object_list[:300]
# for r, l
inds = [8,9,10]

dct = {
   'paragraphs':[o for o in shorter_obj_list if o[0] in inds],
   'root': (4,root_lookup[4][1],root_lookup[4][2]),
   'lemma': (inds[0],lemma_lookup[inds[0]][1],lemma_lookup[inds[0]][2])
}
dict_entry.parse_obj(dct)

# {'paragraphs': [(9, 
#    Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=274320, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=9, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['your (sg.) (only with certain nouns such as those which refer to close family members)'], run_bold=[None], para_text='your (sg.) (only with certain nouns such as those which refer to close family members)')), (10, Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=274320, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=10, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['ton, ta, tes (seulement pour certains substantifs, tels ceux qui se rapportent aux membres de la proche famille)'], run_bold=[None], para_text='ton, ta, tes (seulement pour certains substantifs, tels ceux qui se rapportent aux membres de la proche famille)')), 
#    (11, Docx_Paragraph_and_Runs(run_italic=[None, True, None], para_left_indent=182880, run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt'], paragraph_enumeration=11, run_font_size_pt=[None, None, 8.0], para_first_line_indent=-91440, run_text=['aan  ', 'prn,ind  ', 'DFZH  [an]:Z<->'], run_bold=[True, None, None], para_text='aan  prn,ind  DFZH  [an]:Z<->'))], 'root': [(4, [True], ['A'])], 'lemma': (5, [True, False, False], ['a  ', 'prn,sbj,sf  ', 'DFZH  Z<->'])
#    } did not have the key: paragraphs"

NameError: name 'root_lookup' is not defined